# Chapter 7: Ensemble Learning and Random Forests

Pada chapter ini, kita mempelajari **Ensemble Learning**, yaitu pendekatan Machine Learning yang menggabungkan beberapa model untuk menghasilkan prediksi yang lebih akurat dan stabil dibandingkan model tunggal.

Ide utama dari Ensemble Learning adalah bahwa **sekumpulan model yang beragam** sering kali mampu memberikan prediksi yang lebih baik daripada satu model terbaik sekalipun.

Pendekatan ini banyak digunakan pada tahap akhir proyek Machine Learning dan menjadi fondasi berbagai algoritma canggih yang sering memenangkan kompetisi Machine Learning.

## 1. Ensemble Learning: Intuisi Dasar

Ensemble Learning terinspirasi dari konsep **wisdom of the crowd**, yaitu gagasan bahwa keputusan kolektif dari banyak individu sering kali lebih baik dibandingkan keputusan satu ahli.

Dalam konteks Machine Learning:
- setiap model disebut sebagai *predictor* atau *learner*,
- sekumpulan predictor disebut sebagai **ensemble**,
- prediksi akhir diperoleh dengan menggabungkan prediksi semua predictor.

Jika masing-masing model memiliki performa yang cukup baik dan membuat kesalahan yang berbeda, maka kesalahan tersebut dapat saling menutupi.

### 1.1 Weak Learners vs Strong Learners

Dalam Ensemble Learning, sering kali digunakan **weak learners**, yaitu model yang performanya hanya sedikit lebih baik daripada tebakan acak.

Meskipun setiap model secara individual lemah, penggabungan banyak weak learners dapat menghasilkan **strong learner** dengan performa tinggi.

Kunci keberhasilan pendekatan ini adalah **diversitas**, yaitu setiap model membuat jenis kesalahan yang berbeda.

### 1.2 Mengapa Ensemble Bisa Lebih Baik?

Secara intuitif:
- jika beberapa model membuat kesalahan secara bersamaan, ensemble tetap bisa salah,
- namun jika kesalahan tiap model tidak berkorelasi kuat, ensemble cenderung memilih prediksi yang benar melalui mekanisme agregasi.

Oleh karena itu, Ensemble Learning bekerja paling efektif ketika model-model di dalamnya **cukup akurat dan saling independen**.

Chapter ini akan membahas beberapa teknik ensemble populer, antara lain:
- Voting Classifiers,
- Bagging dan Pasting,
- Random Forests,
- Boosting (AdaBoost dan Gradient Boosting),
- serta Stacking.

Seluruh teknik ini akan dijelaskan secara bertahap dengan intuisi, contoh kode, dan interpretasi hasil.

## 2. Voting Classifiers

Salah satu bentuk Ensemble Learning yang paling sederhana adalah **Voting Classifier**.

Ide utamanya adalah melatih beberapa classifier yang berbeda, kemudian menggabungkan prediksi mereka untuk menentukan prediksi akhir.

Voting Classifier sangat efektif ketika model-model penyusunnya cukup akurat dan membuat kesalahan yang berbeda.

### 2.1 Hard Voting

Pada **hard voting**, setiap classifier memberikan satu suara untuk kelas yang diprediksinya.

Kelas yang memperoleh suara terbanyak akan dipilih sebagai prediksi akhir ensemble.

Pendekatan ini sederhana dan bekerja dengan baik jika semua classifier memiliki performa yang relatif seimbang.

Berikut contoh penerapan hard voting menggunakan tiga classifier berbeda:
- Logistic Regression
- Random Forest
- Support Vector Machine

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

voting_clf

Voting Classifier di atas akan mengambil prediksi kelas dari masing-masing model, lalu menentukan kelas mayoritas sebagai output akhir.

### 2.2 Training and Evaluating the Voting Classifier

Untuk mengevaluasi performa Voting Classifier, kita akan melatih masing-masing model dan membandingkan akurasinya.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

Hasil evaluasi biasanya menunjukkan bahwa Voting Classifier memiliki akurasi yang **setara atau lebih baik** dibandingkan masing-masing model individual.

Hal ini menunjukkan kekuatan agregasi prediksi dalam Ensemble Learning.

### 2.3 Soft Voting

Pada **soft voting**, setiap classifier memberikan probabilitas untuk setiap kelas.

Probabilitas tersebut kemudian dirata-ratakan, dan kelas dengan probabilitas rata-rata tertinggi dipilih sebagai prediksi akhir.

Soft voting biasanya memberikan performa yang lebih baik dibandingkan hard voting, terutama jika model-model memiliki tingkat kepercayaan yang berbeda.

Agar soft voting dapat digunakan, setiap classifier harus mampu menghasilkan probabilitas kelas.

Sebagai contoh, Support Vector Machine perlu diinisialisasi dengan `probability=True`.

In [ ]:
svm_clf = SVC(probability=True)

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft'
)

voting_clf_soft

Soft voting mempertimbangkan tingkat keyakinan masing-masing model, sehingga sering kali menghasilkan prediksi yang lebih stabil dan akurat.

## 3. Bagging and Pasting

Voting Classifier menggabungkan model yang dilatih pada dataset yang sama. Pendekatan lain untuk meningkatkan **diversitas model** adalah dengan melatih setiap model pada **subset data yang berbeda**.

Pendekatan ini dikenal sebagai **Bagging** dan **Pasting**.

### 3.1 Bagging (Bootstrap Aggregating)

**Bagging** adalah teknik ensemble yang melatih beberapa model pada subset data training yang diambil **dengan pengembalian (with replacement)**.

Artinya, satu instance data bisa muncul lebih dari satu kali dalam satu subset, sementara instance lain mungkin tidak muncul sama sekali.

Pendekatan ini efektif untuk mengurangi **variance**, terutama pada model yang tidak stabil seperti Decision Tree.

### 3.2 Pasting

**Pasting** mirip dengan Bagging, tetapi subset data diambil **tanpa pengembalian (without replacement)**.

Akibatnya, setiap subset berisi instance yang unik. Pasting cenderung menghasilkan model yang sedikit kurang beragam dibandingkan Bagging.

Baik Bagging maupun Pasting bertujuan untuk melatih banyak model yang berbeda, kemudian menggabungkan prediksinya melalui voting (classification) atau averaging (regression).

### 3.3 Bagging with Scikit-Learn

Scikit-Learn menyediakan class `BaggingClassifier` dan `BaggingRegressor` untuk menerapkan teknik ini secara langsung.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1
)

bag_clf

Pada konfigurasi di atas:
- `n_estimators=500` berarti 500 Decision Tree dilatih,
- `max_samples=100` menentukan jumlah instance pada setiap subset,
- `bootstrap=True` mengaktifkan Bagging (jika False → Pasting).

Parameter `n_jobs=-1` memungkinkan training paralel menggunakan semua core CPU.

### 3.4 Training and Evaluating Bagging Classifier

Kita dapat melatih Bagging Classifier dan membandingkan performanya dengan Decision Tree tunggal.

In [ ]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

Biasanya, Bagging menghasilkan akurasi yang lebih tinggi dan lebih stabil dibandingkan Decision Tree tunggal.

Hal ini terjadi karena Bagging secara signifikan mengurangi variance model.

## 4. Out-of-Bag (OOB) Evaluation

Salah satu keunggulan utama teknik Bagging adalah kemampuannya untuk melakukan evaluasi model **tanpa memerlukan validation set terpisah**.

Pendekatan ini dikenal sebagai **Out-of-Bag (OOB) evaluation**.

### 4.1 Apa itu Out-of-Bag Samples?

Pada Bagging, setiap model dilatih menggunakan subset data yang diambil dengan pengembalian (*bootstrap sampling*).

Akibatnya, sekitar **63%** dari data training digunakan untuk melatih setiap estimator, sementara **sekitar 37% sisanya tidak ikut terpilih**.

Instance yang tidak terpilih ini disebut sebagai **out-of-bag samples**.

Karena setiap instance training kemungkinan besar akan menjadi out-of-bag untuk beberapa estimator, kita dapat menggunakan prediksi dari estimator-estimator tersebut untuk mengevaluasi performa model.

Dengan kata lain, OOB evaluation memberikan estimasi performa yang mirip dengan cross-validation, tetapi **lebih efisien secara komputasi**.

### 4.2 Mengaktifkan OOB Evaluation di Scikit-Learn

Untuk menggunakan OOB evaluation, kita hanya perlu mengatur parameter `oob_score=True` saat membuat BaggingClassifier.

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    oob_score=True
)

bag_clf.fit(X_train, y_train)

Setelah model dilatih, kita dapat mengakses skor OOB melalui atribut `oob_score_`.

In [ ]:
bag_clf.oob_score_

Nilai OOB score ini memberikan estimasi akurasi model pada data yang tidak digunakan saat training setiap estimator.

Dalam banyak kasus, OOB score sangat dekat dengan hasil evaluasi pada test set.

## 5. Random Forests

**Random Forest** merupakan salah satu algoritma ensemble paling populer dan kuat.

Secara konsep, Random Forest adalah kumpulan **Decision Tree** yang dilatih menggunakan teknik **Bagging**, dengan tambahan sumber diversitas berupa **pemilihan fitur secara acak** pada setiap split.

### 5.1 Mengapa Random Forest Bekerja dengan Baik?

Decision Tree tunggal cenderung memiliki variance yang tinggi. Random Forest mengurangi masalah ini dengan:
- melatih banyak pohon pada subset data yang berbeda (bootstrap sampling),
- membatasi jumlah fitur yang dipertimbangkan pada setiap split.

Kombinasi ini menghasilkan pohon-pohon yang **kurang berkorelasi**, sehingga prediksi gabungannya menjadi lebih stabil dan akurat.

### 5.2 Random Forest for Classification

Untuk tugas klasifikasi, Random Forest menggabungkan prediksi setiap pohon menggunakan **majority voting**.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)

rnd_clf.fit(X_train, y_train)

Model Random Forest di atas terdiri dari 500 Decision Tree dengan jumlah leaf node dibatasi untuk mencegah overfitting.

Parameter `n_jobs=-1` memungkinkan training dilakukan secara paralel.

### 5.3 Evaluating the Random Forest

Setelah training, kita dapat mengevaluasi performa Random Forest pada test set.

In [ ]:
y_pred_rf = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred_rf)

Biasanya, Random Forest memberikan performa yang lebih baik dibandingkan Decision Tree tunggal karena variance yang lebih rendah.

### 5.4 Random Forest vs Bagging

Random Forest pada dasarnya adalah **Bagging Decision Trees** dengan tambahan randomisasi fitur.

Perbedaan utama:
- Bagging mempertimbangkan semua fitur pada setiap split,
- Random Forest hanya mempertimbangkan subset fitur secara acak.

Randomisasi fitur ini meningkatkan diversitas antar pohon.

## 6. Extra-Trees (Extremely Randomized Trees)

Selain Random Forest, terdapat varian ensemble berbasis Decision Tree lain yang disebut **Extra-Trees** atau **Extremely Randomized Trees**.

Extra-Trees memperkenalkan tingkat randomisasi yang lebih tinggi dibandingkan Random Forest, sehingga menghasilkan model yang lebih beragam.

### 6.1 Perbedaan Extra-Trees dan Random Forest

Perbedaan utama antara Extra-Trees dan Random Forest adalah:
- Random Forest mencari split terbaik di antara subset fitur acak,
- Extra-Trees memilih threshold split **secara acak**, tanpa mencari nilai terbaik.

Akibatnya, Extra-Trees biasanya:
- lebih cepat dilatih,
- memiliki bias sedikit lebih tinggi,
- tetapi variance yang lebih rendah.

### 6.2 Training an Extra-Trees Classifier

Scikit-Learn menyediakan class `ExtraTreesClassifier` untuk mengimplementasikan metode ini.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

extra_trees_clf = ExtraTreesClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)

extra_trees_clf.fit(X_train, y_train)

Model Extra-Trees di atas dilatih dengan konfigurasi yang mirip dengan Random Forest untuk memudahkan perbandingan performa.

### 6.3 Evaluating Extra-Trees

Kita dapat mengevaluasi performa Extra-Trees pada test set.

In [ ]:
y_pred_extra = extra_trees_clf.predict(X_test)
accuracy_score(y_test, y_pred_extra)

Dalam banyak kasus, Extra-Trees dapat memberikan performa yang sebanding atau bahkan sedikit lebih baik dibandingkan Random Forest, dengan waktu training yang lebih singkat.

## 6.4 Feature Importance

Sama seperti Random Forest, Extra-Trees juga dapat digunakan untuk mengestimasi **feature importance**.

Feature importance menunjukkan seberapa besar kontribusi setiap fitur dalam mengurangi impurity di seluruh pohon.

In [ ]:
feature_importances = rnd_clf.feature_importances_
feature_importances

Nilai feature importance ini berguna untuk:
- memahami fitur mana yang paling berpengaruh,
- melakukan feature selection,
- serta meningkatkan interpretabilitas model ensemble.

## 7. Boosting

Berbeda dengan Bagging yang melatih model secara independen, **Boosting** melatih model **secara berurutan**.

Setiap model baru berfokus untuk memperbaiki kesalahan yang dibuat oleh model sebelumnya.

Dengan pendekatan ini, Boosting mampu mengubah sekumpulan **weak learners** menjadi model yang sangat kuat.

### 7.1 Intuisi Boosting

Secara intuitif:
- model pertama dilatih pada seluruh data,
- model berikutnya memberikan bobot lebih besar pada instance yang salah diprediksi,
- proses ini berlanjut hingga terbentuk ensemble yang fokus pada kasus-kasus sulit.

Boosting sangat efektif untuk mengurangi **bias**, tetapi rentan terhadap **overfitting** jika data sangat noisy.

## 7.2 AdaBoost (Adaptive Boosting)

**AdaBoost** merupakan salah satu algoritma Boosting paling awal dan populer.

Pada AdaBoost, setiap instance data memiliki bobot. Bobot instance yang salah diklasifikasikan akan ditingkatkan, sehingga model berikutnya lebih fokus pada instance tersebut.

### 7.2.1 Training an AdaBoost Classifier

Secara default, AdaBoost menggunakan Decision Tree dengan kedalaman 1 (*decision stump*) sebagai weak learner.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    learning_rate=0.5,
    random_state=42
)

ada_clf.fit(X_train, y_train)

Parameter penting pada AdaBoost:
- `n_estimators`: jumlah weak learners,
- `learning_rate`: seberapa besar kontribusi setiap model baru.

Learning rate yang kecil biasanya membutuhkan lebih banyak estimator, tetapi dapat meningkatkan generalisasi.

### 7.2.2 Evaluating AdaBoost

Setelah training, kita dapat mengevaluasi performa AdaBoost pada test set.

In [ ]:
y_pred_ada = ada_clf.predict(X_test)
accuracy_score(y_test, y_pred_ada)

Dalam banyak kasus, AdaBoost mampu mengungguli Bagging dan Random Forest pada dataset tertentu, terutama ketika pola data relatif sederhana.

## 7.3 Gradient Boosting

**Gradient Boosting** merupakan pendekatan Boosting yang lebih umum dan fleksibel.

Alih-alih menyesuaikan bobot instance secara eksplisit, Gradient Boosting melatih model baru untuk **memprediksi residual (kesalahan)** dari model sebelumnya.

### 7.3.1 Intuisi Gradient Boosting

Proses Gradient Boosting:
1. model pertama memprediksi target,
2. residual dihitung sebagai selisih antara target dan prediksi,
3. model berikutnya dilatih untuk memprediksi residual tersebut,
4. prediksi model-model digabungkan secara bertahap.

Pendekatan ini mirip dengan optimisasi menggunakan *gradient descent*.

### 7.3.2 Training a Gradient Boosting Regressor

Contoh berikut menunjukkan penggunaan Gradient Boosting untuk tugas regresi.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

gbrt.fit(X_reg, y_reg.ravel())

Gradient Boosting sangat powerful, tetapi juga sensitif terhadap hyperparameter.

Regularization yang baik (misalnya dengan learning rate kecil dan early stopping) sangat penting untuk mencegah overfitting.

## 8. Stacking

**Stacking** (stacked generalization) merupakan teknik ensemble yang lebih lanjut dibandingkan voting, bagging, atau boosting.

Alih-alih hanya menggabungkan prediksi secara langsung, stacking menggunakan **model tambahan (meta-learner)** untuk mempelajari cara terbaik mengombinasikan prediksi dari beberapa model dasar.

### 8.1 Intuisi Stacking

Pada stacking:
- beberapa model dasar (*base learners*) dilatih pada data training,
- setiap model menghasilkan prediksi,
- prediksi-prediksi tersebut digunakan sebagai fitur input untuk model tingkat kedua (*meta-learner*).

Meta-learner kemudian belajar bagaimana menimbang kontribusi masing-masing model dasar.

Pendekatan ini memungkinkan ensemble untuk:
- memanfaatkan keunggulan setiap model dasar,
- mengurangi kelemahan individual model,
- menghasilkan prediksi yang lebih adaptif.

Namun, stacking juga lebih kompleks dan berisiko overfitting jika tidak dirancang dengan hati-hati.

### 8.2 Stacking in Practice

Scikit-Learn menyediakan implementasi stacking melalui class `StackingClassifier` dan `StackingRegressor`.

Pada praktiknya, meta-learner sering menggunakan model sederhana seperti Logistic Regression untuk menjaga generalisasi.

Meskipun tidak selalu memberikan peningkatan performa yang signifikan, stacking sering digunakan dalam kompetisi Machine Learning dan sistem produksi berskala besar ketika setiap peningkatan kecil sangat bernilai.

## Closing Summary (Chapter 7)

Chapter 7 membahas berbagai teknik **Ensemble Learning** yang bertujuan meningkatkan performa dan stabilitas model Machine Learning.

Konsep-konsep utama yang dipelajari meliputi:
- Voting Classifiers (hard dan soft voting),
- Bagging dan Out-of-Bag evaluation,
- Random Forest dan Extra-Trees,
- Boosting (AdaBoost dan Gradient Boosting),
- serta Stacking sebagai teknik ensemble tingkat lanjut.

Melalui chapter ini, kita memahami bahwa menggabungkan banyak model yang beragam sering kali menghasilkan sistem yang lebih kuat dibandingkan model tunggal.

Pemahaman Ensemble Learning ini menjadi fondasi penting untuk mempelajari model Machine Learning modern yang lebih kompleks dan berperforma tinggi pada chapter berikutnya.